# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
df.isnull().sum()

loan_size           0
interest_rate       0
homeowner           0
borrower_income     0
debt_to_income      0
num_of_accounts     0
derogatory_marks    0
total_debt          0
loan_status         0
dtype: int64

In [5]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
77531    False
77532     True
77533    False
77534    False
77535    False
Length: 77536, dtype: bool

# Split the Data into Training and Testing

In [6]:
from sklearn.model_selection import train_test_split
# Create our features
# YOUR CODE HERE
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X.drop("homeowner", axis=1, inplace=True)
X.head()
# Create our target
# YOUR CODE HERE
y = df["total_debt"].values.ravel()
y[:5]

array([22800, 13600, 16100, 22700, 23000], dtype=int64)

In [7]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [8]:
# Check the balance of our target values


In [9]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs', random_state=1)

In [14]:
log_reg.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [15]:
y_pred = log_reg.predict(X_test)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.013011152416356878

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00         1
       1500       0.00      0.00      1.00      0.00      0.00      0.00         1
       1700       0.00      0.00      1.00      0.00      0.00      0.00         1
       2000       0.00      0.00      1.00      0.00      0.00      0.00         1
       2300       0.00      0.00      1.00      0.00      0.00      0.00         2
       2700       0.00      0.00      1.00      0.00      0.00      0.00         1
       2900       0.00      0.00      1.00      0.00      0.00      0.00         1
       3000       0.00      0.00      1.00      0.00      0.00      0.00         2
       3300       0.00      0.00      1.00      0.00      0.00      0.00         3
       3400       0.00      0.00      1.00      0.00      0.00      0.00         2
       3500       0.00      0.00      1.00      0.00      0.00      0.00         1
   

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [19]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({15400: 501,
         15100: 501,
         21100: 501,
         19800: 501,
         10900: 501,
         13100: 501,
         25900: 501,
         10700: 501,
         20500: 501,
         17100: 501,
         26200: 501,
         19500: 501,
         12000: 501,
         22800: 501,
         12100: 501,
         13500: 501,
         23400: 501,
         31800: 501,
         20100: 501,
         13400: 501,
         19000: 501,
         16300: 501,
         11000: 501,
         14300: 501,
         25100: 501,
         12400: 501,
         19300: 501,
         18300: 501,
         20600: 501,
         22600: 501,
         14500: 501,
         16700: 501,
         23100: 501,
         13000: 501,
         14900: 501,
         11900: 501,
         17200: 501,
         23500: 501,
         25500: 501,
         14700: 501,
         17000: 501,
         18200: 501,
         20700: 501,
         63000: 501,
         21000: 501,
         20200: 501,
         11300: 501,
         5880

In [20]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
log_reg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

0.013011152416356878

In [22]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [23]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00         1
       1500       0.00      0.00      1.00      0.00      0.00      0.00         1
       1700       0.00      0.00      1.00      0.00      0.00      0.00         1
       2000       0.00      0.00      1.00      0.00      0.00      0.00         1
       2300       0.00      0.00      1.00      0.00      0.00      0.00         2
       2700       0.00      0.00      1.00      0.00      0.00      0.00         1
       2900       0.00      0.00      1.00      0.00      0.00      0.00         1
       3000       0.00      0.00      1.00      0.00      0.00      0.00         2
       3300       0.00      0.00      1.00      0.00      0.00      0.00         3
       3400       0.00      0.00      1.00      0.00      0.00      0.00         2
       3500       0.00      0.00      1.00      0.00      0.00      0.00         1
   

### SMOTE Oversampling

In [ ]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE
from collections import Counter

X_resampled= SMOTE().fit_resample(X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
